In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
data = pd.read_parquet('../data/yellow_tripdata_2022-01.parquet')

In [3]:
# 1
print("Question 1: {}".format(data.shape[1]))

Question 1: 19


In [4]:
data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [5]:
data['duration'] = data.tpep_dropoff_datetime - data.tpep_pickup_datetime
data['duration'] = data['duration'].apply(lambda td: td.total_seconds() / 60)

In [6]:
# 2
print("Question 2: {}".format(round(data.duration.std(), 2)))

Question 2: 46.45


In [7]:
# 3
init_count = len(data)
data = data[(data.duration<=60) & (data.duration>=1)]
print("Question 3: {}".format(round(len(data) / init_count * 100)))

Question 3: 98


In [8]:
categorical = ['PULocationID', 'DOLocationID']

data[categorical] = data[categorical].astype(str)

In [9]:
train_dicts = data[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [10]:
# 4
print("Question 4: {}".format(round(X_train.shape[1])))

Question 4: 515


In [11]:
# 5
target = 'duration'
y_train = data[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

print("Question 5: {}".format(round(mean_squared_error(y_train, y_pred, squared=False), 2)))

Question 5: 6.99


In [12]:
del data
val = pd.read_parquet('../data/yellow_tripdata_2022-02.parquet')
val['duration'] = val.tpep_dropoff_datetime - val.tpep_pickup_datetime
val['duration'] = val['duration'].apply(lambda td: td.total_seconds() / 60)
val = val[(val['duration']<=60) & (val['duration']>=1)]
val[categorical] = val[categorical].astype(str)

In [13]:
# 6
val_pred = lr.predict(dv.transform(val[categorical].to_dict(orient='records')))
print("Question 6: {}".format(round(mean_squared_error(val['duration'], val_pred, squared=False), 2)))


Question 6: 7.79
